# Executing Summary:
I will build a first version of recommender based on the correlation between wine and its rating. Recommender system can be build with three consecutive steps: Constructing Pivot Table, Creating a Sparse Matrix, and Calculating Cosine similarities. It should be noted that this will be a basic recommender that only focuses on wine ratings. 


In [1]:
# Import Required Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances

In [2]:
# Reading csv file
wine_df = pd.read_csv('/home/docode/project/EDA on Collected Data/new_wine.csv', index_col = 0) 

rating_df = wine_df.copy()
rating_df = pd.DataFrame(wine_df['wine rating'])
rating_df

,wine rating
0,3.4
1,3.4
2,3.1
3,3.4
4,3.4
...,...
22201,4.1
22202,4.2
22203,3.9
22204,3.8


In [3]:
# Create a new column by joining three columns, this helps to search the wine of interest 
columns = ['wine name', 'winery', 'wine year']
wine_df['wine name, winery and year'] = wine_df[columns].apply(lambda x: ' '.join(x.values.astype(str)), axis = 1)

# Rearranging the columns 
cols = wine_df.columns.tolist()
cols = cols[0:3] + cols[-1:] + cols[3:-1]
cols

wine_df = wine_df[cols]
wine_df = wine_df.fillna("") 
wine_df

,wine name,winery,wine year,wine description,"wine name, winery and year",wine rating,wine price,wine type,wine region,wine country,grape information,wine acidity,wine intensity,wine sweetness,wine tannin
0,Cabernet Sauvignon,Carta Vieja,2019,Cabernet Sauvignon is the most widely grown gr...,Cabernet Sauvignon Carta Vieja 2019,3.4,4.99,Red,Loncomilla Valley,Chile,Cabernet Sauvignon,3.04375,3.78178,1.77275,3.16434
1,Merlot,Carta Vieja,2019,Merlot is a staple of the wine producing regio...,Merlot Carta Vieja 2019,3.4,4.99,Red,Loncomilla Valley,Chile,Merlot,2.02042,3.48272,1.94994,2.36619
2,Cabernet Sauvignon,Three Wishes,N.V.,"Known as the king of red wine grapes, Cabernet...",Cabernet Sauvignon Three Wishes N.V.,3.1,4.99,Red,California,United States,Cabernet Sauvignon,3.20616,4.54563,1.96201,3.56995
3,Cabernet Sauvignon,Crane Lake,2016,"Known as the king of red wine grapes, Cabernet...",Cabernet Sauvignon Crane Lake 2016,3.4,4.99,Red,California,United States,Cabernet Sauvignon,3.0142,4.73893,1.74318,3.54043
4,Pinot Noir,Crane Lake,2016,Pinot Noir has the well deserved reputation of...,Pinot Noir Crane Lake 2016,3.4,4.99,Red,California,United States,Pinot Noir,3.40543,2.8322,1.50087,2.14787
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22201,10 Years Old White Port,Quinta de Santa Eufémia,N.V.,,10 Years Old White Port Quinta de Santa Eufémi...,4.1,29.99,Fortified,Porto,Portugal,Gouveio,,,,
22202,Late Bottled Vintage Port,Quinta do Crasto,2015,,Late Bottled Vintage Port Quinta do Crasto 2015,4.2,27.48,Fortified,Porto,Portugal,Touriga Nacional,,,,
22203,Pedro Ximenez 1827 Jerez-Xeres-Sherry,Osborne,N.V.,,Pedro Ximenez 1827 Jerez-Xeres-Sherry Osborne ...,3.9,17.59,Fortified,Pedro Ximénez Sherry (PX),Spain,Pedro Ximenez,2.87102,4.82332,4.947,
22204,Lagrima Fine White Port,Krohn,N.V.,,Lagrima Fine White Port Krohn N.V.,3.8,12.52,Fortified,Porto,Portugal,Gouveio,,,,


In [4]:
# Now we redefine the wine dataframe column and leave only four columns below
cols = ['wine name, winery and year', 'wine rating', 'wine price', 'wine region']
wine_df = wine_df[cols]
wine_df = wine_df.rename(columns = {'wine name, winery and year': 'wine_label'})
wine_df

,wine_label,wine rating,wine price,wine region
0,Cabernet Sauvignon Carta Vieja 2019,3.4,4.99,Loncomilla Valley
1,Merlot Carta Vieja 2019,3.4,4.99,Loncomilla Valley
2,Cabernet Sauvignon Three Wishes N.V.,3.1,4.99,California
3,Cabernet Sauvignon Crane Lake 2016,3.4,4.99,California
4,Pinot Noir Crane Lake 2016,3.4,4.99,California
...,...,...,...,...
22201,10 Years Old White Port Quinta de Santa Eufémi...,4.1,29.99,Porto
22202,Late Bottled Vintage Port Quinta do Crasto 2015,4.2,27.48,Porto
22203,Pedro Ximenez 1827 Jerez-Xeres-Sherry Osborne ...,3.9,17.59,Pedro Ximénez Sherry (PX)
22204,Lagrima Fine White Port Krohn N.V.,3.8,12.52,Porto


In [5]:
# Read previously defined user rating dataset
wine_ratings = pd.read_csv('/home/docode/project/Constructing User Rating Dataset for Recommendation System/User_rating.csv', index_col = [0])

# Insert Wine_id column
wine_df.insert(0, 'wine_id', wine_df.index + 1)
wine_ratings.insert(0, 'wine_id', wine_ratings.index + 1 )

# Melt wine dataframe  
wine = pd.melt(wine_df, id_vars=['wine_label'],var_name="wine_style",value_name="wine_id")
print(wine.head(100))

# Melt ratings dataframe
ratings = pd.melt(wine_ratings, id_vars = ['wine_id'], var_name='user',value_name="rating")
ratings

                                           wine_label wine_style wine_id
0                 Cabernet Sauvignon Carta Vieja 2019    wine_id       1
1                             Merlot Carta Vieja 2019    wine_id       2
2                Cabernet Sauvignon Three Wishes N.V.    wine_id       3
3                  Cabernet Sauvignon Crane Lake 2016    wine_id       4
4                          Pinot Noir Crane Lake 2016    wine_id       5
..                                                ...        ...     ...
95                               Malbec Barefoot 2016    wine_id      96
96                     Flat Bed Red Blend Fetzer 2017    wine_id      97
97                      Eagle Peak Merlot Fetzer 2015    wine_id      98
98                               Merlot Barefoot 2015    wine_id      99
99  Firebaugh's Ferry Pinot Noir Butterfield Stati...    wine_id     100

[100 rows x 3 columns]


,wine_id,user,rating
0,1,user-0,3.5
1,2,user-0,NaN
2,3,user-0,3.2
3,4,user-0,3.6
4,5,user-0,3.3
...,...,...,...
21955995,21977,user-999,NaN
21955996,21978,user-999,3.9
21955997,21979,user-999,NaN
21955998,21980,user-999,3.9


# Constructing Pivot Table

In [6]:
# Merge wine and ratings dataframes 
recommendation = pd.merge(wine, ratings, on = 'wine_id')

# Pivot the table
pivot = pd.pivot_table(recommendation, index='wine_label', columns='user', values = 'rating')

print('Shape of a Pivot Table:', pivot.shape)



Shape of a Pivot Table: (20985, 1000)


In [7]:
# pivot = pd.pivot_table(recommendation, index='wine_label', columns='user', values = 'rating')
# pivot.head()

# print('Shape of a Pivot Table:', pivot.shape)


### Creating a Sparse Matrix, Calculating Cosine Similarity and Building a Recommender:

In [8]:
# Creating a Sparse Matrix
sparse_pivot=sparse.csr_matrix(pivot.fillna(0.0))
print('Sparse Matrix Shape:',sparse_pivot.shape)

# Calculate Cosine Similarity
recommender = pairwise_distances(sparse_pivot, metric = 'cosine')

# Building a Recommender 
recommender_df = pd.DataFrame(recommender, columns = pivot.index, index = pivot.index)
recommender_df

Sparse Matrix Shape: (20985, 1000)


wine_label,Chardonnay Clone 4 Cambria 2015,#42 Cabernet Sauvignon Spoken Barrel 2016,'3C Champagne Bourgeois-Diaz N.V.,'61 Franciacorta Nature Berlucchi 2010,'61 Franciacorta Nature Berlucchi 2012,'Askos' Verdeca Li Veli 2018,'Askos' Verdeca Li Veli 2019,'B Blanc de Blancs Champagne Bourgeois-Diaz N.V.,'Giulio Ferrari' Riserva del Fondatore Ferrari 2006,'Giulio Ferrari' Riserva del Fondatore Ferrari 2007,...,Ŕoxa Itxas Harri 2018,Žan Keltis 2018,ΓΙΟΡΤΗ (Feast) White Seméli 2019,Κουμανταρία (Commandaria) Tσiakkaς 2012,Μοσχοφίλερο (Moschofilero) Κτήμα Σπυροπούλου (Domaine Spiropoulos) 2017,Ξυνιστέρι (Xynisteri) Tσiakkaς 2019,Игристое Полусладкое (Sparkling Semi-Sweet) Советское Шампанское N.V.,‘61 Franciacorta Brut Berlucchi 2017,‘61 Franciacorta Rosé Berlucchi N.V.,‘61 Franciacorta Satèn Berlucchi 1961
wine_label,,,,,,,,,,,,,,,,,,,,,
Chardonnay Clone 4 Cambria 2015,0.000000,0.511377,0.517296,0.528972,0.417493,0.406296,0.504525,0.520763,0.432766,0.547037,...,0.432766,0.387598,0.526322,0.415747,0.556865,0.542199,0.492762,0.557175,0.561209,0.529679
#42 Cabernet Sauvignon Spoken Barrel 2016,0.511377,0.000000,0.483472,0.531330,0.397659,0.405075,0.491363,0.501965,0.417820,0.547542,...,0.426250,0.404617,0.505669,0.411976,0.549300,0.496285,0.503934,0.495854,0.524847,0.504715
'3C Champagne Bourgeois-Diaz N.V.,0.517296,0.483472,0.000000,0.502811,0.356511,0.375073,0.488718,0.502120,0.385062,0.508251,...,0.398632,0.363483,0.468823,0.367154,0.497404,0.480871,0.509498,0.482175,0.481905,0.509240
'61 Franciacorta Nature Berlucchi 2010,0.528972,0.531330,0.502811,0.000000,0.393114,0.379274,0.481785,0.538276,0.391847,0.544110,...,0.401162,0.381402,0.531444,0.407522,0.512202,0.521499,0.509747,0.518089,0.518068,0.534738
'61 Franciacorta Nature Berlucchi 2012,0.417493,0.397659,0.356511,0.393114,0.000000,0.192966,0.376143,0.419508,0.240431,0.400865,...,0.263232,0.240740,0.368329,0.252989,0.404581,0.381465,0.386689,0.374381,0.393661,0.391640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ξυνιστέρι (Xynisteri) Tσiakkaς 2019,0.542199,0.496285,0.480871,0.521499,0.381465,0.390932,0.534723,0.508165,0.386514,0.553127,...,0.418322,0.385295,0.498372,0.411866,0.532109,0.000000,0.552362,0.513016,0.541593,0.498506
Игристое Полусладкое (Sparkling Semi-Sweet) Советское Шампанское N.V.,0.492762,0.503934,0.509498,0.509747,0.386689,0.386422,0.484861,0.539532,0.393614,0.519183,...,0.411524,0.381692,0.490439,0.397755,0.524431,0.552362,0.000000,0.517977,0.473470,0.499452
‘61 Franciacorta Brut Berlucchi 2017,0.557175,0.495854,0.482175,0.518089,0.374381,0.392822,0.482328,0.503943,0.394429,0.538801,...,0.413921,0.369667,0.498626,0.410831,0.499733,0.513016,0.517977,0.000000,0.493870,0.504582


# Testing Result

In [10]:
from random import randrange

# From the Sparse Matrix shape we can see how many unique entries does matrix have. Based on this you can generate random number between 0 and unique wine numbers
random_index = randrange(20985)

search = recommender_df.index[random_index] 

print('Name of the Wine you have selected:', search)

for wine_label in wine_df.loc[wine_df['wine_label'].str.contains(search), 'wine_label']:

    print('')
    print('10 closest wine labels based on Wine Rating')
    print('')
    print(recommender_df[wine_label].sort_values(ascending = False)[1:11])
    print('')
    print('*******************************************************************************************')
    print('')

Name of the Wine you have selected: Vintner's Reserve Riesling Kendall-Jackson 2016

10 closest wine labels based on Wine Rating

wine_label
Vintage Port Dow's 2017                                              0.556156
Loupiac Château de Ricaud 2009                                       0.554916
Gota de Arena Tridente 2018                                          0.553980
Pinot Gris Jules Taylor 2018                                         0.552170
La Serra Barolo Roberto Voerzio 2011                                 0.551439
Broquel Cabernet Sauvignon Trapiche 2018                             0.551228
Pago de Santa Cruz Gran Reserva Ribera del Duero Viña Sastre 2012    0.548733
Falanghina Beneventano Vesevo 2017                                   0.547247
Chardonnay Indaba 2018                                               0.547194
Coteau de Fontenay Chablis Patrick Piuze 2018                        0.546882
Name: Vintner's Reserve Riesling Kendall-Jackson 2016, dtype: float64

********

# End of Notebook:

This notebook showed how to build a basic recommendation system based on the correlation between wine and it's rating. Matrix factorization technique was used to make a meaningful recommendations. Next Notebook will focus on Wine Description 